In [ ]:
# fitting model using SVM
from matchcleaner import unpack, mergeclean
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn import metrics
import pandas as pd
import numpy as np
import seaborn as sns
# run unzip filename in terminal
import zipfile
import json
import os
# pd.set_option('display.max_columns', None)

In [ ]:
# load data files
# matches_df = pd.read_json("/workspaces/project-9/sample_data/sample_data/d2_250_1_matches.jsonl", lines=True)
# timelines_df = pd.read_json("/workspaces/project-9/sample_data/sample_data/d2_250_1_timelines.jsonl", lines=True)
# print(matches_df['metadata'].head())

0    {'dataVersion': '2', 'matchId': 'NA1_542195455...
1    {'dataVersion': '2', 'matchId': 'NA1_541777086...
2    {'dataVersion': '2', 'matchId': 'NA1_542134877...
3    {'dataVersion': '2', 'matchId': 'NA1_542030874...
4    {'dataVersion': '2', 'matchId': 'NA1_542009850...
Name: metadata, dtype: object


In [8]:
df = mergeclean("/workspaces/project-9/sample_data/sample_data/d2_250_1_timelines.jsonl", "/workspaces/project-9/sample_data/sample_data/d2_250_1_matches.jsonl")
df = unpack(df)
print(df.head())

   deaths  dragonKills  baronKills  firstBloodKill  firstBloodAssist  kills  \
0       8            0           0           False             False      3   
1      12            1           0           False             False      6   
2       5            0           0           False             False      4   
3       6            0           0           False             False      3   
4       7            0           0           False             False      0   

     lane  neutralMinionsKilled  turretTakedowns  turretsLost  visionScore  \
0     TOP                     4                1           11           12   
1  JUNGLE                   143                0           11           27   
2  MIDDLE                     8                3           11           22   
3  BOTTOM                     0                0           11           19   
4  MIDDLE                     0                0           11           59   

   goldEarned  timeCCingOthers  enemyMissingPings  clear

In [ ]:
X = df[['deaths', 'dragonKills', 'baronKills', 'firstBloodKill', 'firstBloodAssist', 'kills']].copy()
X['firstBloodKill'] = df[['firstBloodKill']].astype(int)
X['firstBloodAssist'] = df[['firstBloodAssist']].astype(int)
# target = outcome of game
y = df[['win']].astype(int)

deaths              int64
dragonKills         int64
baronKills          int64
firstBloodKill      int64
firstBloodAssist    int64
kills               int64
dtype: object


In [43]:
# split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 180)

In [44]:
# scaling X
scale = StandardScaler()
X_train_scaled = scale.fit_transform(X_train)
X_test_scaled = scale.fit_transform(X_test)

In [38]:
clf = svm.SVC(kernel = 'rbf')
# train svm
clf.fit(X_train_scaled, y_train)
y_pred = clf.predict(X_test_scaled)
print("SVM Accuracy:", metrics.accuracy_score(y_test, y_pred))

SVM Accuracy: 0.7350427350427351


/home/codespace/.local/lib/python3.12/site-packages/sklearn/utils/validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
# -1 <- negative correlation, 1 <- positive correlation
# calculated corr coef of all X events and whether the player wins
# visualizes what is most and least significant in Accuracy score
corr = X.join(y).corr()['win'].sort_values(ascending=False)
print(corr)

win                 1.000000
kills               0.280959
baronKills          0.174465
dragonKills         0.148256
firstBloodKill      0.014030
firstBloodAssist   -0.006964
deaths             -0.324169
Name: win, dtype: float64


In [ ]:
# Conclusion:
# A SVM accuracy of 0.73 for predicting player w/l based on in-game events
# means the X values (in-game events) correlate with winning.
# From calculating the correlation coefficients of all X events and whether the player wins,
# it's predicted that kills have the strong correlation with winning, and deaths has a stronger
# negative correlation.